In [1]:
import os, json, random
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import StepLR
import numpy as np
from datetime import datetime
from transformers import (
    Wav2Vec2FeatureExtractor,
    HubertModel,
)
from textgrids import TextGrid
from tqdm import tqdm

/home/xingxing/anaconda3/envs/vitscn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pinyin_list = ['a', 'ai', 'an', 'ang', 'ao', 'ba', 'bai', 'ban', 'bang', 'bao', 'be', 'bei', 'ben', 'beng', 'bi', 'bian', 'bianr', 'biao', 'bie', 'bin', 'bing', 'bo', 'bu', 'ca', 'cai', 'can', 'cang', 'cao', 'ce', 'cen', 'ceng', 'cha', 'chai', 'chan', 'chang', 'chao', 'che', 'chen', 'cheng', 'chi', 'chong', 'chou', 'chu', 'chua', 'chuai', 'chuan', 'chuang', 'chui', 'chun', 'chuo', 'ci', 'cong', 'cou', 'cu', 'cuan', 'cui', 'cun', 'cuo', 'da', 'dai', 'dan', 'dang', 'dao', 'de', 'dei', 'den', 'deng', 'di', 'dia', 'dian', 'diao', 'die', 'ding', 'diu', 'dong', 'dou', 'du', 'duan', 'dui', 'dun', 'duo', 'e', 'ei', 'en', 'ng', 'eng', 'er', 'fa', 'fan', 'fang', 'fei', 'fen', 'feng', 'fo', 'fou', 'fu', 'ga', 'gai', 'gan', 'gang', 'gao', 'ge', 'gei', 'gen', 'geng', 'gong', 'gou', 'gu', 'gua', 'guai', 'guan', 'guang', 'gui', 'gun', 'guo', 'ha', 'hai', 'hair', 'han', 'hang', 'hao', 'he', 'hei', 'hen', 'heng', 'hong', 'hou', 'hu', 'hua', 'huai', 'huan', 'huang', 'hui', 'huir', 'hun', 'huo', 'ji', 'jia', 'jian', 'jiang', 'jiao', 'jie', 'jin', 'jing', 'jiong', 'jiu', 'ju', 'juan', 'jue', 'jun', 'ka', 'kai', 'kan', 'kang', 'kao', 'ke', 'kei', 'ken', 'keng', 'kong', 'kou', 'ku', 'kua', 'kuai', 'kuan', 'kuang', 'kui', 'kun', 'kuo', 'la', 'lai', 'lan', 'lang', 'lao', 'le', 'lei', 'leng', 'li', 'lia', 'lian', 'liang', 'liao', 'lie', 'lin', 'ling', 'liu', 'lo', 'long', 'lou', 'lu', 'lv', 'luan', 'lve', 'lue', 'lun', 'luo', 'ma', 'mai', 'man', 'mang', 'mao', 'me', 'mei', 'men', 'meng', 'mi', 'mian', 'miao', 'mie', 'min', 'ming', 'miu', 'mo', 'mou', 'mu', 'na', 'nar', 'nai', 'nan', 'nang', 'nao', 'ne', 'nei', 'nen', 'neng', 'ni', 'nia', 'nian', 'niang', 'niao', 'nie', 'nin', 'ning', 'niu', 'nong', 'nou', 'nu', 'nv', 'nuan', 'nve', 'nue', 'nuo', 'o', 'ou', 'pa', 'pai', 'pan', 'pang', 'pao', 'pe', 'pei', 'pen', 'peng', 'pi', 'pian', 'piao', 'pie', 'pin', 'ping', 'po', 'pou', 'pu', 'qi', 'qia', 'qian', 'qiang', 'qiao', 'qie', 'qin', 'qing', 'qiong', 'qiu', 'qu', 'quan', 'que', 'qun', 'ran', 'rang', 'rao', 're', 'ren', 'reng', 'ri', 'rong', 'rou', 'ru', 'rua', 'ruan', 'rui', 'run', 'ruo', 'sa', 'sai', 'san', 'sang', 'sao', 'se', 'sen', 'seng', 'sha', 'shai', 'shan', 'shang', 'shao', 'she', 'shei', 'shen', 'sheng', 'shi', 'shir', 'shou', 'shu', 'shua', 'shuai', 'shuan', 'shuang', 'shui', 'shun', 'shuo', 'si', 'song', 'sou', 'su', 'suan', 'sui', 'sun', 'suo', 'ta', 'tai', 'tan', 'tang', 'tao', 'te', 'tei', 'teng', 'ti', 'tian', 'tianr', 'tiao', 'tie', 'ting', 'tong', 'tou', 'tu', 'tuan', 'tui', 'tun', 'tuo', 'wa', 'wai', 'wan', 'wanr', 'wang', 'wei', 'weir', 'wen', 'weng', 'wo', 'wu', 'xi', 'xia', 'xian', 'xiang', 'xiao', 'xie', 'xin', 'xing', 'xiong', 'xiu', 'xu', 'xuan', 'xue', 'xun', 'ya', 'yan', 'yang', 'yao', 'ye', 'yi', 'yin', 'ying', 'yo', 'yong', 'you', 'yu', 'yuan', 'yue', 'yun', 'za', 'zai', 'zan', 'zang', 'zao', 'ze', 'zei', 'zen', 'zeng', 'zha', 'zhai', 'zhan', 'zhang', 'zhao', 'zhe', 'zhei', 'zhen', 'zheng', 'zher', 'zhi', 'zhong', 'zhou', 'zhu', 'zhua', 'zhuai', 'zhuan', 'zhuang', 'zhui', 'zhun', 'zhuo', 'zi', 'zong', 'zou', 'zu', 'zuan', 'zui', 'zun', 'zuo', 'gair', 'yir', 'jinr', 'menr', 'her', 'gunr', 'huanr', 'jiar', 'duir', 'dir', 'banr', 'yanr', 'cir', 'genr', 'ger', 'tongr', 'renr', 'kuair', 'yuanr', 'liar', 'dunr', 'fur', 'zaor', 'yangr', 'ter', 'yingr', 'fanr', 'tuor', 'pir', 'jingr', 'hanr', 'duor', 'tanr', 'zhunr', 'jir', 'huor', 'far', 'dianr', 'zhur', 'tour', 'qir', 'mingr', 'lanr', 'lir', 'daor', 'niur']

pinyin_to_label = {p: i for i, p in enumerate(pinyin_list)}
label_to_pinyin = {i: p for i, p in enumerate(pinyin_list)}

In [3]:

def change_layers(model):
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(512, len(pinyin_list))
    return model

model = models.resnet18(pretrained=False)
change_layers(model)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
model.load_state_dict(torch.load('../checkpoints/07_09_2023_14:46:23/ckp_18.pt')['model_state_dict'])

<All keys matched successfully>

In [5]:
model.eval()
model = model.cuda()

In [6]:
hubert_model_path = "TencentGameMate/chinese-hubert-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hubert_model_path)
hubert_model = HubertModel.from_pretrained(hubert_model_path)

hubert_model = hubert_model.cuda()
hubert_model.eval()

HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertGroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): HubertNoLayerNormConvLayer(

In [7]:
metadata = json.load(open('test_py_list.json'))

metadata[0]

{'file_name': '008002',
 'line_text': '喜欢把李敖的书插在牛仔裤的前面？',
 'gt_py': 'xi3 huan1 ba2 li3 ao2 de5 shu1 cha1 zai4 niu2 zai3 ku4 de5 qian2 mian4',
 'line_py': ['xi3',
  'huan5',
  'ba3',
  'li3',
  'ao2',
  'de5',
  'shu1',
  'cha1',
  'zai4',
  'niu2',
  'zai3',
  'ku4',
  'de5',
  'qian2',
  'mian4'],
 'hetero_py': [['xi3'],
  ['huan1'],
  ['ba3', 'ba4'],
  ['li3'],
  ['ao2'],
  ['de5', 'di2', 'di4'],
  ['shu1'],
  ['cha1'],
  ['zai4'],
  ['niu2'],
  ['zi1', 'zi3', 'zai3'],
  ['ku4'],
  ['de5', 'di2', 'di4'],
  ['qian2'],
  ['mian4']],
 'g2pm_py': ['xi3',
  'huan1',
  'ba3',
  'li3',
  'ao2',
  'de5',
  'shu1',
  'cha1',
  'zai4',
  'niu2',
  'zai3',
  'ku4',
  'de5',
  'qian2',
  'mian4']}

In [8]:
file_keys = [m['file_name'] for m in metadata]

file_keys[:2]

['008002', '008003']

In [9]:

grid_dir = './biaobei_aligned'
grids = os.listdir(grid_dir)

py_list = []

for g in grids:
    file_name = g.split('.')[0]
    if file_name not in file_keys:
        continue
    grid = TextGrid(os.path.join(grid_dir, g))    
    pds = []
    for w in grid['words']:
        # if len(w.text) > 0 and w.xmax - w.xmin > 0.7:
        #     print(g, w.text, w.xmin, w.xmax)            
        if len(w.text) > 0 and w.xmax - w.xmin > 0.03 and w.xmax - w.xmin < 0.6:
            pds.append((w.text, w.xmin, w.xmax))
        elif w.text != '':
            print(g, w)
    py_list.append({
        'file_name': file_name,
        'pinyin_clips': pds
    })
    

008493.TextGrid <Interval text="shuai4" xmin=1.77 xmax=1.79>
008233.TextGrid <Interval text="hang2" xmin=1.21 xmax=1.23>
008612.TextGrid <Interval text="o2" xmin=6.58 xmax=6.606>
008557.TextGrid <Interval text="he2" xmin=3.44 xmax=3.46>
009219.TextGrid <Interval text="o2" xmin=0.32 xmax=0.35>
009860.TextGrid <Interval text="a1" xmin=5.15 xmax=5.18>
009649.TextGrid <Interval text="mu3" xmin=0.99 xmax=1.01>
008648.TextGrid <Interval text="luo1" xmin=0.51 xmax=0.53>


In [10]:
audio_py_pairs = {}
for p in py_list:
    file_path = 'lab_wav_pairs/' +  p['file_name'] + '.wav'
    for pc in p['pinyin_clips']:
        if file_path in audio_py_pairs:
            audio_py_pairs[file_path].append((pc[0], pc[1], pc[2]))
        else:
            audio_py_pairs[file_path] = [(pc[0], pc[1], pc[2])]

In [11]:
audio_py_hetero_list = {}
for m in metadata:
    filename = 'lab_wav_pairs/' + m['file_name'] + '.wav'
    m_hetero_py = []
    for hp in m['hetero_py']:
        pys = [p[:-1] for p in hp]
        m_hetero_py.append(pys)
    audio_py_hetero_list[filename] = {
        'nm_hetero_py': m_hetero_py,
        'm_hetero_py': m['hetero_py']
    }
audio_py_hetero_list.keys()

dict_keys(['lab_wav_pairs/008002.wav', 'lab_wav_pairs/008003.wav', 'lab_wav_pairs/008004.wav', 'lab_wav_pairs/008005.wav', 'lab_wav_pairs/008006.wav', 'lab_wav_pairs/008007.wav', 'lab_wav_pairs/008008.wav', 'lab_wav_pairs/008009.wav', 'lab_wav_pairs/008010.wav', 'lab_wav_pairs/008011.wav', 'lab_wav_pairs/008012.wav', 'lab_wav_pairs/008013.wav', 'lab_wav_pairs/008014.wav', 'lab_wav_pairs/008015.wav', 'lab_wav_pairs/008016.wav', 'lab_wav_pairs/008017.wav', 'lab_wav_pairs/008018.wav', 'lab_wav_pairs/008019.wav', 'lab_wav_pairs/008020.wav', 'lab_wav_pairs/008021.wav', 'lab_wav_pairs/008022.wav', 'lab_wav_pairs/008023.wav', 'lab_wav_pairs/008024.wav', 'lab_wav_pairs/008025.wav', 'lab_wav_pairs/008027.wav', 'lab_wav_pairs/008028.wav', 'lab_wav_pairs/008029.wav', 'lab_wav_pairs/008030.wav', 'lab_wav_pairs/008031.wav', 'lab_wav_pairs/008032.wav', 'lab_wav_pairs/008033.wav', 'lab_wav_pairs/008034.wav', 'lab_wav_pairs/008035.wav', 'lab_wav_pairs/008036.wav', 'lab_wav_pairs/008037.wav', 'lab_wav_

In [12]:
audio_py_hetero_list['lab_wav_pairs/008002.wav']

{'nm_hetero_py': [['xi'],
  ['huan'],
  ['ba', 'ba'],
  ['li'],
  ['ao'],
  ['de', 'di', 'di'],
  ['shu'],
  ['cha'],
  ['zai'],
  ['niu'],
  ['zi', 'zi', 'zai'],
  ['ku'],
  ['de', 'di', 'di'],
  ['qian'],
  ['mian']],
 'm_hetero_py': [['xi3'],
  ['huan1'],
  ['ba3', 'ba4'],
  ['li3'],
  ['ao2'],
  ['de5', 'di2', 'di4'],
  ['shu1'],
  ['cha1'],
  ['zai4'],
  ['niu2'],
  ['zi1', 'zi3', 'zai3'],
  ['ku4'],
  ['de5', 'di2', 'di4'],
  ['qian2'],
  ['mian4']]}

In [13]:

correct = 0
total = 0
err_list = []

for file_path in tqdm(list(audio_py_pairs.keys())):
    for index, ap in enumerate(audio_py_pairs[file_path]):
        py, start, end = ap
        try:
            SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(file_path, int(start * 48000), int((end-start) * 48000))
            SPEECH_WAVEFORM = torchaudio.functional.resample(SPEECH_WAVEFORM, SAMPLE_RATE, 16000)

            SPEECH_WAVEFORM = F.pad(SPEECH_WAVEFORM, (0, 16000-SPEECH_WAVEFORM.shape[1]), mode='constant', value=0)

            with torch.no_grad():
                # features, _ = hubert_model.extract_features(SPEECH_WAVEFORM.cuda())
                input_values = feature_extractor(SPEECH_WAVEFORM[0], return_tensors="pt", sampling_rate=16000).input_values
                outputs = hubert_model(input_values.cuda())
                last_hidden_state = outputs.last_hidden_state
            
            # print(last_hidden_state.shape)
            with torch.no_grad():
                output = model(torch.unsqueeze(last_hidden_state, 0))
            _, predicted = torch.topk(output, 1)
            # print(predicted, label_to_pinyin[predicted[0][0].item()], py)
            c_sum = sum([1 if label_to_pinyin[predicted[0][i].item()] == py[:-1] else 0 for i in range(len(predicted))])
            correct += c_sum
            if c_sum == 0:
                for i in range(len(predicted)):
                    if py[:-1] != label_to_pinyin[predicted[0][i].item()]:
                        print(file_path, index, start, end, label_to_pinyin[predicted[0][i].item()], py[:-1])
                        err_list.append((file_path, index, start, end, label_to_pinyin[predicted[0][i].item()], py[:-1]))
            total += 1
        except Exception as e:
            print(file_path, ap, e)
            continue

print('Accuracy of the model: %.3f %%' %((100*correct)/(total)))


  0%|          | 0/1943 [00:00<?, ?it/s]2023-09-07 18:57:53.454750: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-07 18:57:53.497307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 18:57:54.293172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/xingxing/anaconda3/envs/vitscn/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do

lab_wav_pairs/008895.wav 6 1.8 2.0 min ming


  1%|          | 18/1943 [00:05<06:47,  4.72it/s]

lab_wav_pairs/009856.wav 18 4.22 4.37 hu hun


  1%|          | 24/1943 [00:06<04:19,  7.38it/s]

...

lab_wav_pairs/009531.wav 5 1.33 1.42 men me


100%|█████████▉| 1942/1943 [07:43<00:00,  4.08it/s]

lab_wav_pairs/009240.wav 5 1.51 1.65 ya jia


100%|██████████| 1943/1943 [07:43<00:00,  4.19it/s]

Accuracy of the model: 98.443 %


In [14]:

detected_err_list = []

for err in err_list:
    filename, index, start, end, predicted, gt = err
    print(predicted, gt)
    print(filename, audio_py_hetero_list[filename]['nm_hetero_py'][index], audio_py_hetero_list[filename]['m_hetero_py'][index])
    nm_hetero_py_i = audio_py_hetero_list[filename]['nm_hetero_py'][index]
    m_hetero_py_i = audio_py_hetero_list[filename]['m_hetero_py'][index]
    if predicted in nm_hetero_py_i:
        hetero_index = nm_hetero_py_i.index(predicted)
        print('!!!')
        audio_py_pairs[filename][index] = (m_hetero_py_i[hetero_index], audio_py_pairs[filename][index][1], audio_py_pairs[filename][index][2])
        detected_err_list.append(err)
    

min ming
lab_wav_pairs/008895.wav ['ming'] ['ming2']
hu hun
lab_wav_pairs/009856.wav ['hun'] ['hun2']
xing xin
lab_wav_pairs/008661.wav ['xin', 'shen'] ['xin4', 'shen1']
de di
lab_wav_pairs/008729.wav ['di', 'de'] ['di4', 'de5']
!!!
de di
lab_wav_pairs/008517.wav ['di', 'de'] ['di4', 'de5']
!!!
ni nei
lab_wav_pairs/009398.wav ['na', 'nei', 'na', 'ne'] ['na3', 'nei3', 'na5', 'ne2']
ng shuai
lab_wav_pairs/008225.wav ['shuai', 'lv'] ['shuai4', 'lv4']
en deng
lab_wav_pairs/009451.wav ['deng'] ['deng3']
ao le
lab_wav_pairs/009149.wav ['le', 'liao'] ['le5', 'liao3']
dei de
lab_wav_pairs/009638.wav ['de', 'dei', 'de'] ['de2', 'dei3', 'de5']
!!!
chan zha
lab_wav_pairs/008360.wav ['cha', 'zha'] ['cha2', 'zha1']
de zha
lab_wav_pairs/008360.wav ['cha', 'zha'] ['cha2', 'zha1']
a ta
lab_wav_pairs/008360.wav ['ta'] ['ta1']
lou lu
lab_wav_pairs/008124.wav ['lou', 'lu'] ['lou4', 'lu4']
!!!
zhang chang
lab_wav_pairs/009676.wav ['chang', 'zhang'] ['chang2', 'zhang3']
!!!
liao le
lab_wav_pairs/009077.wav

In [17]:
g2pa_pairs = {}

for ap in audio_py_pairs:
    file_name = ap.split('/').pop().split('.')[0]
    py_list = ' '.join([a[0] for a in audio_py_pairs[ap]])
    print(file_name, py_list)
    g2pa_pairs[file_name] = py_list
    

009419 guai4 bu4 de2 gu3 ren2 chang2 shuo1 jin4 zhu1 zhe3 chi4 wo3 xian4 zai4 que4 shi2 yue4 lai2 yue4 xiang4 zhu3 ren2 le5
008493 shi4 shi2 shang4 gao1 li2 hun1 yi3 jing1 gei3 ke4 luo2 di4 ya4 de5 she4 hui4 dai4 lai2 le5 hen3 da4 de5 ying2 xiang3
009112 yuan2 lai2 ta1 duo3 qi3 lai2 wan2 wei1 bo2 yu3 you3 ren2 da4 tan2 xing1 zuo4
009334 bei4 cheng1 zan4 le5 zhen1 kai1 xin1 ya1
008763 hei1 zuo4 fang1 nei4 zheng4 zai4 fen1 zhuang1 jia3 mao4 shui3 ni2
009497 shu4 ren2 xiong1 ke3 xi1 wan3 sheng1 le5 yi4 bai3 nian2 huo4 zhe3 zao3 sheng1 le5 yi4 bai3 nian2
009668 wu2 yi2 tang2 jun4 de5 shuai2 shou3 zou3 ren2 dui4 lian2 you2 wang3 luo4 lai2 shuo1 shi4 xue3 shang4 jia1 shuang1
009872 cao2 hong2 liang4 shuo1 hai2 you3 ji3 ge4 kong1 zhe5 mei2 guan1 xi4 rang4 wo3 chao1 chao1
008073 ge4 ti3 gong1 shang1 hu4 gen1 ju4 jing1 ying2 xu1 yao4 ke2 yi3 pin4 qing3 ruo4 gan1 bang1 shou3 huo4 zhe3 xue2 tu2
008822 shuang1 fang1 shang1 ding4 ji4 xu4 jiu4 ci3 wen4 ti2 bao3 chi2 cuo1 shang1
008846 ping2 shi2 wei

In [18]:

json_object = json.dumps(g2pa_pairs, indent=4)

with open("g2pa_pairs.json", "w") as outfile:
    outfile.write(json_object)